In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

%matplotlib inline

pd.set_option('display.max_columns', 50)

In [2]:
data=pd.read_excel('edited final.xlsx',parse_dates=['case_received_date','decision_date'])

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

data = reduce_mem_usage(data)

Mem. usage decreased to 69.26 Mb (6.7% reduction)


In [4]:
data.head()

,agent_state,application_type,case_no,case_received_date,case_status,class_of_admission,country_of_citizenship,decision_date,employer_num_employees,employer_yr_estab,employer_city,employer_name,employer_state,foreign_worker_info_education,foreign_worker_info_major,job_experience,job_title,job_training,job_work_state,pv_unit_of_pay_9089,previous_salary,level,source_name,sector,wage_offer,wage_unit_of_pay
0,NaN,PERM,A-07323-97014,NaT,Certified,J-1,ARMENIA,2012-02-01,NaN,NaN,NEW YORK,NETSOFT USA INC.,NY,NaN,NaN,NaN,NaN,NaN,NY,yr,75629.00,Level II,OES,IT,75629,yr
1,NaN,PERM,A-07332-99439,NaT,Denied,B-2,POLAND,2011-12-21,NaN,NaN,CARLSTADT,PINNACLE ENVIRONEMNTAL CORP,NY,NaN,NaN,NaN,NaN,NaN,NY,yr,37024.00,Level I,OES,Other Economic Sector,37024,yr
2,NaN,PERM,A-07333-99643,NaT,Certified,H-1B,INDIA,2011-12-01,NaN,NaN,GLEN ALLEN,"SCHNABEL ENGINEERING, INC.",VA,NaN,NaN,NaN,NaN,NaN,MD,yr,47923.00,Level I,OES,Aerospace,47923,yr
3,NaN,PERM,A-07339-01930,NaT,Certified,B-2,SOUTH KOREA,2011-12-01,NaN,NaN,FLUSHING,EBENEZER MISSION CHURCH,NY,NaN,NaN,NaN,NaN,NaN,NY,hr,10.97,Level II,OES,Other Economic Sector,10.97,hr
4,NaN,PERM,A-07345-03565,NaT,Certified,L-1,CANADA,2012-01-26,NaN,NaN,ALBANY,ALBANY INTERNATIONAL CORP.,NY,NaN,NaN,NaN,NaN,NaN,NY,yr,94890.00,Level IV,OES,Advanced Mfg,100000,yr


In [5]:
data.source_name.value_counts()

OES                   341918
Other                  25851
CBA                     3854
Employer Conducted       539
SCA                       77
DBA                       24
Name: source_name, dtype: int64

In [5]:
print('info',data.info())
print('Shape==',data.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374362 entries, 0 to 374361
Data columns (total 26 columns):
agent_state                      208239 non-null object
application_type                 135269 non-null object
case_no                          135269 non-null object
case_received_date               239091 non-null datetime64[ns]
case_status                      374362 non-null object
class_of_admission               351517 non-null object
country_of_citizenship           374302 non-null object
decision_date                    374362 non-null datetime64[ns]
employer_num_employees           239013 non-null float32
employer_yr_estab                238958 non-null float16
employer_city                    374348 non-null object
employer_name                    374350 non-null object
employer_state                   374320 non-null object
foreign_worker_info_education    239053 non-null object
foreign_worker_info_major        219390 non-null object
job_experience                 

In [6]:
data.drop_duplicates(inplace=True) 

In [7]:
data.shape

(365627, 26)

In [12]:
data.case_status.value_counts(1)

Certified            0.486835
Certified-Expired    0.396213
Denied               0.068564
Withdrawn            0.048388
Name: case_status, dtype: float64

### Extracting month and year  from Dates
1. The first 2 numbers in the case_no follow a pattern and depict the year .
2. It has been observed that in each record , either the case_no is available or the case_received_date
3. We will hence extract the year from case_no and impute the case_received_date_year column for its missing values

In [8]:
## We have two date columns:-- case_received_date and decision_date so convert date into month and year and try to get some infernces from that.

data['case_received_date_year']=data.case_received_date.dt.year
data['case_received_date_month']=data.case_received_date.dt.month

data['decision_date_month']=data.decision_date.dt.month
data['decision_date_year']=data.decision_date.dt.year

In [11]:
data.isnull().sum().sum()*100/365627

689.3894597499637

In [9]:
(data.isnull().sum()[data.isna().sum()!=0])*100/365627

agent_state                      44.076887
application_type                 63.003553
case_no                          63.003553
case_received_date               36.996994
class_of_admission                5.749302
country_of_citizenship            0.016410
employer_num_employees           37.018327
employer_yr_estab                37.033370
employer_city                     0.003829
employer_name                     0.003282
employer_state                    0.011487
foreign_worker_info_education    37.007114
foreign_worker_info_major        41.479978
job_experience                   36.999729
job_title                        37.008481
job_training                     37.000550
job_work_state                    0.028171
pv_unit_of_pay_9089               2.633832
previous_salary                   0.605535
level                             7.410558
source_name                       0.573535
sector                           64.699544
wage_offer                       31.390187
wage_unit_o

### Imputing Missing values in columns less with than 2% missing values

In [10]:
data.source_name.fillna(data.source_name.mode()[0],inplace=True)
data.previous_salary.fillna(data.previous_salary.median(),inplace=True)
data.job_work_state.fillna(data.job_work_state.mode()[0],inplace=True)
data.country_of_citizenship.fillna(data.country_of_citizenship.mode()[0],inplace=True)
data.employer_city.fillna(data.employer_city.mode()[0],inplace=True)
data.employer_name.fillna(data.employer_name.mode()[0],inplace=True)


data.isnull().sum()[data.isnull().sum()!=0]

agent_state                      161157
application_type                 230358
case_no                          230358
case_received_date               135271
class_of_admission                21021
employer_num_employees           135349
employer_yr_estab                135404
employer_state                       42
foreign_worker_info_education    135308
foreign_worker_info_major        151662
job_experience                   135281
job_title                        135313
job_training                     135284
pv_unit_of_pay_9089                9630
level                             27095
sector                           236559
wage_offer                       114771
wage_unit_of_pay                 115689
case_received_date_year          135271
case_received_date_month         135271
dtype: int64

### Extract year as we got pattern from case_no and impute it into feature

In [11]:
raw=data.case_no[data.case_no.notnull()]
year=raw.transform(lambda x:x.split("-")[1])
data['Yr']='20'+year
data['Yr'].head()

0    2007323
1    2007332
2    2007333
3    2007339
4    2007345
Name: Yr, dtype: object

In [12]:
y=data['Yr'].astype('str')
new=[]
for i in y.index:
    new.append(y[i][:-3])
    
new[1]

'2007'

In [13]:
data['Received_Year']=new

data.drop('Yr',axis=1,inplace=True)

data.case_received_date_year.fillna(data.Received_Year,inplace=True)



data.decision_date_year=data.decision_date_year.astype('int')
data.head()

,agent_state,application_type,case_no,case_received_date,case_status,class_of_admission,country_of_citizenship,decision_date,employer_num_employees,employer_yr_estab,employer_city,employer_name,employer_state,foreign_worker_info_education,foreign_worker_info_major,job_experience,job_title,job_training,job_work_state,pv_unit_of_pay_9089,previous_salary,level,source_name,sector,wage_offer,wage_unit_of_pay,case_received_date_year,case_received_date_month,decision_date_month,decision_date_year,Received_Year
0,NaN,PERM,A-07323-97014,NaT,Certified,J-1,ARMENIA,2012-02-01,NaN,NaN,NEW YORK,NETSOFT USA INC.,NY,NaN,NaN,NaN,NaN,NaN,NY,yr,75629.00,Level II,OES,IT,75629,yr,2007,NaN,2,2012,2007
1,NaN,PERM,A-07332-99439,NaT,Denied,B-2,POLAND,2011-12-21,NaN,NaN,CARLSTADT,PINNACLE ENVIRONEMNTAL CORP,NY,NaN,NaN,NaN,NaN,NaN,NY,yr,37024.00,Level I,OES,Other Economic Sector,37024,yr,2007,NaN,12,2011,2007
2,NaN,PERM,A-07333-99643,NaT,Certified,H-1B,INDIA,2011-12-01,NaN,NaN,GLEN ALLEN,"SCHNABEL ENGINEERING, INC.",VA,NaN,NaN,NaN,NaN,NaN,MD,yr,47923.00,Level I,OES,Aerospace,47923,yr,2007,NaN,12,2011,2007
3,NaN,PERM,A-07339-01930,NaT,Certified,B-2,SOUTH KOREA,2011-12-01,NaN,NaN,FLUSHING,EBENEZER MISSION CHURCH,NY,NaN,NaN,NaN,NaN,NaN,NY,hr,10.97,Level II,OES,Other Economic Sector,10.97,hr,2007,NaN,12,2011,2007
4,NaN,PERM,A-07345-03565,NaT,Certified,L-1,CANADA,2012-01-26,NaN,NaN,ALBANY,ALBANY INTERNATIONAL CORP.,NY,NaN,NaN,NaN,NaN,NaN,NY,yr,94890.00,Level IV,OES,Advanced Mfg,100000,yr,2007,NaN,1,2012,2007


In [14]:
data.case_received_date_year.unique()

array(['2007', '2008', '2006', '2009', '2011', '2010', '2012', '2013',
       '2005', '2014', 2013.0, 2014.0, 2009.0, 2010.0, 2008.0, 2012.0,
       2015.0, 2006.0, 2007.0, 2011.0, '', 2016.0], dtype=object)

In [15]:
data[data.case_received_date_year=='']

,agent_state,application_type,case_no,case_received_date,case_status,class_of_admission,country_of_citizenship,decision_date,employer_num_employees,employer_yr_estab,employer_city,employer_name,employer_state,foreign_worker_info_education,foreign_worker_info_major,job_experience,job_title,job_training,job_work_state,pv_unit_of_pay_9089,previous_salary,level,source_name,sector,wage_offer,wage_unit_of_pay,case_received_date_year,case_received_date_month,decision_date_month,decision_date_year,Received_Year
188375,FLORIDA,NaN,NaN,NaT,Withdrawn,H-1B,CHILE,2014-12-04,88.0,1999.0,COCONUT GROVE,TRAX USA CORP.,FLORIDA,Bachelor's,MANAGEMENT OF TECHNICAL OPERATIONS,Y,Functional Software Specialist,N,FLORIDA,Year,96325.0,Level IV,OES,NaN,96325,Year,,NaN,12,2014,
198698,CALIFORNIA,NaN,NaN,NaT,Withdrawn,H-1B,BRAZIL,2015-03-06,30000.0,1998.0,NEW YORK,PRICEWATERHOUSECOOPERS,NEW YORK,Master's,BUSINESS ADMINISTRATION (GRADUATED: 05/20/2012),Y,Senior Associate,N,NEW YORK,Year,132558.0,Level IV,OES,NaN,132558,Year,,NaN,3,2015,


In [16]:
data.case_received_date_year.replace("",1996,inplace=True) #1996 is the mode

In [17]:
data.case_received_date_year=data.case_received_date_year.astype('int')

In [18]:
data[data.case_received_date_year>data.decision_date_year]   # This imputation is succesful

,agent_state,application_type,case_no,case_received_date,case_status,class_of_admission,country_of_citizenship,decision_date,employer_num_employees,employer_yr_estab,employer_city,employer_name,employer_state,foreign_worker_info_education,foreign_worker_info_major,job_experience,job_title,job_training,job_work_state,pv_unit_of_pay_9089,previous_salary,level,source_name,sector,wage_offer,wage_unit_of_pay,case_received_date_year,case_received_date_month,decision_date_month,decision_date_year,Received_Year


In [19]:
data.employer_yr_estab.isnull().sum()

135404

### Inferences :
1. We have extracted useful data from case_no (63% missing values) and case_received_date (37% missing values) , so now we will drop them.

2. The new extracted features: case_received_date_mn nad decision_date_mn are not contributing to be helpful and moreover have missing values , so we have dropped them.


In [20]:
data.employer_yr_estab.median()

1996.0

### Binning the years 

In [21]:
pd.qcut(data.employer_yr_estab,3)

0                      NaN
1                      NaN
2                      NaN
3                      NaN
4                      NaN
5                      NaN
6                      NaN
7                      NaN
8                      NaN
9                      NaN
10                     NaN
11                     NaN
12                     NaN
13                     NaN
14                     NaN
15                     NaN
16                     NaN
17                     NaN
18                     NaN
19                     NaN
20                     NaN
21                     NaN
22                     NaN
23                     NaN
24                     NaN
25                     NaN
26                     NaN
27                     NaN
28                     NaN
29                     NaN
                ...       
374332    (-0.001, 1988.0]
374333    (2001.0, 2016.0]
374334    (2001.0, 2016.0]
374335    (1988.0, 2001.0]
374336    (2001.0, 2016.0]
374337    (-0.001, 1988.0]
3

In [22]:
data.employer_yr_estab.fillna(-1,inplace=True)  # Keeping the null values aside

In [23]:
def age(x):
    if x==-1:
        return "Unavailable"
    elif x>-1 and x<1600:
        return "Old"
    elif x>1988 and x<2001:
        return 'Old'
    elif x<1988:
        return "Ancient"
    else:
        return "New"
    
data.employer_yr_estab=data.employer_yr_estab.apply(age)

In [24]:
data.employer_yr_estab.value_counts(1)

Unavailable    0.370334
New            0.228637
Ancient        0.207824
Old            0.193205
Name: employer_yr_estab, dtype: float64

In [25]:
135404/3

45134.666666666664

In [26]:
45134*3

135402

In [27]:
state_abb = { 'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR','California': 'CA','Colorado': 'CO',
                 'Connecticut': 'CT','Delaware': 'DE','Florida': 'FL','Georgia': 'GA','Hawaii': 'HI','Idaho': 'ID',
                  'Illinois': 'IL','Indiana': 'IN','Iowa': 'IA','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA',
                'Maine': 'ME','Maryland': 'MD', 'Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS',
                'Missouri': 'MO','Montana': 'MT','Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ',
                'New Mexico': 'NM','New York': 'NY','North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH','Oklahoma': 'OK',
                'Oregon': 'OR','Pennsylvania': 'PA','Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD',
                'Tennessee': 'TN','Texas': 'TX','Utah': 'UT','Vermont': 'VT','Virginia': 'VA','Washington': 'WA',
                'West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY','Northern Mariana Islands':'MP', 'Palau': 'PW', 
                'Puerto Rico': 'PR', 'Virgin Islands': 'VI', 'District of Columbia': 'DC','MARSHALL ISLANDS':'MH'}
us_state_abbrev = {k.upper(): v for k, v in state_abb.items()}
data['agent_state'].replace(us_state_abbrev, inplace=True)
data.agent_state = data.agent_state.astype(str)

In [28]:
data.agent_state.unique()

array(['nan', 'WI', 'NC', 'CO', 'CA', 'MI', 'NY', 'TX', 'PA', 'NJ', 'MO',
       'FL', 'VA', 'OH', 'DC', 'IN', 'GA', 'MA', 'IL', 'AL', 'OK', 'WA',
       'CT', 'MN', 'AR', 'HI', 'MD', 'UT', 'PR', 'AZ', 'NV', 'NH', 'OR',
       'KS', 'NM', 'VT', 'TN', 'NE', 'LA', 'KY', 'GUAM', 'IA', 'SC', 'RI',
       'ME', 'ID', 'ND', 'AK', 'MS', 'DE', 'MH', 'SD', 'MT', 'WV', 'GU',
       'MP', 'WY', 'VI'], dtype=object)

In [29]:
def region(x):
    return regions[x]

data['agent_state'] = data['agent_state'].replace({
    'ME':'Northeast', 
    'NY':'Northeast', 
    'NJ':'Northeast', 
    'VT':'Northeast',
    'MA':'Northeast',
    'RI':'Northeast',
    'CT':'Northeast',
    'NH':'Northeast',
    'PA':'Northeast',
    'IL':'Midwest',
    'IN':'Midwest',
    'IA':'Midwest', 
    'KS':'Midwest',
    'MI':'Midwest', 
    'MN':'Midwest', 
    'MO':'Midwest', 
    'NE':'Midwest',
    'ND':'Midwest', 
    'OH':'Midwest', 
    'SD':'Midwest', 
    'WI':'Midwest',
    'DE':'South',
    'FL':'South', 
    'GA':'South',
    'MD':'South',
    'NC':'South',
    'SC':'South',
    'VA':'South',
    'WV':'South',
    'AL':'South',
    'KY':'South',
    'MS':'South',
    'TN':'South',
    'AR':'South',
    'LA':'South', 
    'OK':'South',
    'TX':'South',
    'MT':'West',
    'WY':'West', 
    'CO':'West', 
    'NM':'West', 
    'ID':'West', 
    'UT':'West', 
    'AZ':'West', 
    'NV':'West',
    'WA':'West', 
    'OR':'West', 
    'CA':'West', 
    'AK':'West',
    'HI':'West',
    'DC':'Other', 
    'GUAM':'Island',
    'GU':'Island',
    'PR':'Island', 
    'MP':'Island', 
    'VI':'Island',
    'BC':'Northeast',
    'BRITISH COLUMBIA':'Northeast',  
    'nan':'Other',
    'FEDERATED STATES OF MICRONESIA':'Other',
'MH':'Island'})


In [30]:
data.agent_state.value_counts()

Other        165132
West          65564
Northeast     53072
South         52972
Midwest       28620
Island          267
Name: agent_state, dtype: int64

In [31]:
data.agent_state.isnull().sum()

0

### Correcting the values of units of pay

In [32]:
data.pv_unit_of_pay_9089.replace('yr','Year',inplace=True)
data.pv_unit_of_pay_9089.replace('hr','Hour',inplace=True)
data.pv_unit_of_pay_9089.replace('bi','Bi-Weekly',inplace=True)
data.pv_unit_of_pay_9089.replace('mth','Month',inplace=True)
data.pv_unit_of_pay_9089.replace('wk','Week',inplace=True)


data.wage_unit_of_pay.replace('hr','Hour',inplace=True)
data.wage_unit_of_pay.replace('bi','Bi-Weekly',inplace=True)
data.wage_unit_of_pay.replace('mth','Month',inplace=True)
data.wage_unit_of_pay.replace('wk','Week',inplace=True)
data.wage_unit_of_pay.replace('yr','Year',inplace=True)

data.pv_unit_of_pay_9089.unique(),data.wage_unit_of_pay.unique()

(array(['Year', 'Hour', nan, 'Bi-Weekly', 'Month', 'Week'], dtype=object),
 array(['Year', 'Hour', 'Month', 'Week', 'Bi-Weekly', nan], dtype=object))

### Analyzing columns like states to generalize them

In [33]:
data.agent_state.unique()

array(['Other', 'Midwest', 'South', 'West', 'Northeast', 'Island'],
      dtype=object)

In [34]:
data.agent_state.value_counts().head(),data.agent_state.value_counts(1).head()

(Other        165132
 West          65564
 Northeast     53072
 South         52972
 Midwest       28620
 Name: agent_state, dtype: int64, Other        0.451641
 West         0.179319
 Northeast    0.145153
 South        0.144880
 Midwest      0.078276
 Name: agent_state, dtype: float64)

##### Inference:
1. Since, agent_state has 58 unique values, 44% nan values and it is not contributing significantly to the business problem. 

2. Moreover , similar information is available in 'Source' column.

In [35]:
#2
data.application_type.value_counts()

ONLINE      112564
PERM         20576
MAILEDIN      2129
Name: application_type, dtype: int64

#### Creating a new level for missing values 

In [36]:
## rather than creating new level we devide null values equally into all categories

data.application_type.fillna('unavailable',inplace=True)

In [37]:
data.application_type.isnull().sum()/data.shape[0]

0.0

### Generalizing the Class_of_admsn of Visa

In [38]:
data.class_of_admission.unique()

array(['J-1', 'B-2', 'H-1B', 'L-1', 'EWI', 'E-2', nan, 'E-1', 'H-2B',
       'TPS', 'F-1', 'B-1', 'C-1', 'Not in USA', 'TN', 'H-4', 'O-1',
       'R-1', 'L-2', 'Q', 'F-2', 'H-1B1', 'Parolee', 'G-5', 'E-3', 'H-2A',
       'VWT', 'P-1', 'A1/A2', 'D-1', 'A-3', 'R-2', 'H-1C', 'H-3', 'J-2',
       'P-4', 'I', 'H-1A', 'G-1', 'VWB', 'G-4', 'P-3', 'AOS/H-1B', 'O-3',
       'Parol', 'O-2', 'H1B', 'N', 'T-1', 'TD', 'M-1', 'K-1', 'U-1',
       'AOS', 'P-2', 'C-3', 'V-2', 'M-2'], dtype=object)

Visa categories are further divided into sub-categories. So we can use the parent Category in order to generalize it. 

https://www.bankbazaar.com/visa/types-of-us-visa.html

https://uk.usembassy.gov/visas/visa-directory/

https://travel.state.gov/content/travel/en/us-visas/visa-information-resources/all-visa-categories.html

In [39]:
data.class_of_admission.replace('J-1','J',inplace=True)
data.class_of_admission.replace('J-2','J',inplace=True)
data.class_of_admission.replace('B-2','B',inplace=True)
data.class_of_admission.replace('B-1','B',inplace=True)
data.class_of_admission.replace('H-2B','H-1B',inplace=True)
data.class_of_admission.replace('H-2A','H-1B',inplace=True)
data.class_of_admission.replace('H-1A','H-1B',inplace=True)
data.class_of_admission.replace('H-1B1','H-1B',inplace=True)
data.class_of_admission.replace('H-3','H-1B',inplace=True)
data.class_of_admission.replace('H-4','H-1B',inplace=True)
data.class_of_admission.replace('H-1C','H-1B',inplace=True)
data.class_of_admission.replace('H1B','H-1B',inplace=True)
data.class_of_admission.replace('L-1','L',inplace=True)
data.class_of_admission.replace('L-2','L',inplace=True)
data.class_of_admission.replace('Parol','Parole',inplace=True)
data.class_of_admission.replace('Parolee','Parole',inplace=True)
data.class_of_admission.replace('E-1','E',inplace=True)
data.class_of_admission.replace('E-2','E',inplace=True)
data.class_of_admission.replace('E-3','E',inplace=True)
data.class_of_admission.replace('F-1','F',inplace=True)
data.class_of_admission.replace('F-2','F',inplace=True)
data.class_of_admission.replace('G-1','G',inplace=True)
data.class_of_admission.replace('G-4','G',inplace=True)
data.class_of_admission.replace('G-5','G',inplace=True)
data.class_of_admission.replace('M-1','M',inplace=True)
data.class_of_admission.replace('M-2','M',inplace=True)
data.class_of_admission.replace('O-1','O',inplace=True)
data.class_of_admission.replace('O-2','O',inplace=True)
data.class_of_admission.replace('O-3','O',inplace=True)
data.class_of_admission.replace('P-1','P',inplace=True)
data.class_of_admission.replace('P-2','P',inplace=True)
data.class_of_admission.replace('P-3','P',inplace=True)
data.class_of_admission.replace('P-4','P',inplace=True)
data.class_of_admission.replace('R-1','R',inplace=True)
data.class_of_admission.replace('R-2','R',inplace=True)
data.class_of_admission.replace('C-1','C',inplace=True)
data.class_of_admission.replace('C-2','C',inplace=True)
data.class_of_admission.replace('C-3','C',inplace=True)
data.class_of_admission.replace('A1/A2','A',inplace=True)
data.class_of_admission.replace('A-2','A',inplace=True)
data.class_of_admission.replace('A-3','P',inplace=True)
data.class_of_admission.replace('AOS/H-1B','AOS',inplace=True)

## Creating a new level for missing values(5%)
data.class_of_admission.replace('Not in USA','Unavailable',inplace=True)
data.class_of_admission.replace(np.nan,'Unavailable',inplace=True)

In [40]:
data.class_of_admission.unique()

array(['J', 'B', 'H-1B', 'L', 'EWI', 'E', 'Unavailable', 'TPS', 'F', 'C',
       'TN', 'O', 'R', 'Q', 'Parole', 'G', 'VWT', 'P', 'A', 'D-1', 'I',
       'VWB', 'AOS', 'N', 'T-1', 'TD', 'M', 'K-1', 'U-1', 'V-2'],
      dtype=object)

In [41]:
(data.isnull().sum()[data.isnull().sum()/data.shape[0]>0])

case_no                          230358
case_received_date               135271
employer_num_employees           135349
employer_state                       42
foreign_worker_info_education    135308
foreign_worker_info_major        151662
job_experience                   135281
job_title                        135313
job_training                     135284
pv_unit_of_pay_9089                9630
level                             27095
sector                           236559
wage_offer                       114771
wage_unit_of_pay                 115689
case_received_date_month         135271
dtype: int64

In [42]:
239006/data.shape[0]
# 65.3% rows have same pv_unit_of_pay and wage_unit_of_pay

0.6536880482021294

In [43]:
data.pv_unit_of_pay_9089.unique()

array(['Year', 'Hour', nan, 'Bi-Weekly', 'Month', 'Week'], dtype=object)

In [44]:
data.pv_unit_of_pay_9089.fillna('Year',inplace=True)

In [45]:
data.pv_unit_of_pay_9089.isnull().sum()

0

________________________________________________________________
### According the work permit in US , an average person works for 8 hours a day and 5 days in a week.
#### Considering this, we can convert the unit of pay on a yearly basis so that all values of pv_unit_of_pay are same.
#### We will now convert the units: 
1. No. of working days in a year are approximately around 250 (~ = 365-(52*2))
2. https://www.opm.gov/policy-data-oversight/pay-leave/pay-administration/fact-sheets/computing-hourly-rates-of-pay-using-the-2087-hour-divisor/
3. 
Hourly and biweekly rates of pay for most Federal civilian employees are computed as required by 5 U.S.C. 5504(b). 

Hourly and Bi-weekly conversion - https://www.law.cornell.edu/uscode/text/5/5504

In [46]:
for unit in data.pv_unit_of_pay_9089.unique():
    if unit == "Hour":
        data.loc[data['pv_unit_of_pay_9089'] == unit, 'previous_salary'] = data['previous_salary'].apply(lambda x: float(x) * 8 * 250)
        data.loc[data['pv_unit_of_pay_9089'] == unit, 'pv_unit_of_pay_9089'] = data['pv_unit_of_pay_9089'].replace(to_replace = unit, value = "Year") 
    elif unit == "Week":
        data.loc[data['pv_unit_of_pay_9089'] == unit, 'previous_salary'] = data['previous_salary'].apply(lambda x: float(x) * 50)
        data.loc[data['pv_unit_of_pay_9089'] == unit, 'pv_unit_of_pay_9089'] = data['pv_unit_of_pay_9089'].replace(to_replace = unit, value = "Year")
    elif unit == "Month":
        data.loc[data['pv_unit_of_pay_9089'] == unit, 'previous_salary'] = data['previous_salary'].apply(lambda x: float(x) * 12)
        data.loc[data['pv_unit_of_pay_9089'] == unit, 'pv_unit_of_pay_9089'] = data['pv_unit_of_pay_9089'].replace(to_replace = unit, value = "Year")
    elif unit == "Bi-Weekly":  
        data.loc[data['pv_unit_of_pay_9089'] == unit, 'previous_salary'] = data['previous_salary'].apply(lambda x: float(x) * 40)
        data.loc[data['pv_unit_of_pay_9089'] == unit, 'pv_unit_of_pay_9089'] = data['pv_unit_of_pay_9089'].replace(to_replace = unit, value = "Year")
    
    else:
        continue
        data['previous_salary'] = data.previous_salary.astype(float)


In [47]:
data.wage_offer.fillna(data.previous_salary,inplace=True)

In [48]:
data.employer_state.mode()

0    CALIFORNIA
dtype: object

In [49]:
data.employer_state.fillna('CALIFORNIA',inplace=True)

In [50]:
data.isnull().sum()[data.isna().sum()!=0]

case_no                          230358
case_received_date               135271
employer_num_employees           135349
foreign_worker_info_education    135308
foreign_worker_info_major        151662
job_experience                   135281
job_title                        135313
job_training                     135284
level                             27095
sector                           236559
wage_unit_of_pay                 115689
case_received_date_month         135271
dtype: int64

In [51]:
np.dtype(data.previous_salary)

dtype('float32')

In [52]:
data.previous_salary.median()


86501.0

In [53]:
data[data.wage_offer=="#############"]

,agent_state,application_type,case_no,case_received_date,case_status,class_of_admission,country_of_citizenship,decision_date,employer_num_employees,employer_yr_estab,employer_city,employer_name,employer_state,foreign_worker_info_education,foreign_worker_info_major,job_experience,job_title,job_training,job_work_state,pv_unit_of_pay_9089,previous_salary,level,source_name,sector,wage_offer,wage_unit_of_pay,case_received_date_year,case_received_date_month,decision_date_month,decision_date_year,Received_Year
202865,West,unavailable,NaN,2015-01-14,Certified,H-1B,INDIA,2015-08-18,596.0,New,Jersey City,"Brillio, LLC",NEW JERSEY,Master's,CIVIL ENGINEERING,Y,Computer Programmer II,N,NEW JERSEY,Year,108763.0,Level III,OES,NaN,#############,Year,2015,1.0,8,2015,
209472,Northeast,unavailable,NaN,2015-07-13,Withdrawn,E,AUSTRALIA,2015-08-05,55.0,Old,Tuxedo Park,"Live Technology Holdings, Inc.",NEW YORK,Master's,COMPUTER SCIENCE,Y,Lead Delphi Engineer,N,NEVADA,Year,7036600.0,Level II,OES,NaN,#############,Year,2015,7.0,8,2015,


In [54]:
data.wage_offer.replace("#############",90000,inplace=True)   #90000 is the median of this column

In [55]:
data["wage_offer"][data["wage_offer"]=="#############"]

Series([], Name: wage_offer, dtype: float64)

### The columns previous_salary and wage_offer is the same for more than 95% of the rows. 
#### They are providing the same information. Thus ,we will make a new column which is their mean and drop the original columns after extracting information from them.

In [56]:
data['meansal']=(data.previous_salary+data.wage_offer)/2



data.head()

,agent_state,application_type,case_no,case_received_date,case_status,class_of_admission,country_of_citizenship,decision_date,employer_num_employees,employer_yr_estab,employer_city,employer_name,employer_state,foreign_worker_info_education,foreign_worker_info_major,job_experience,job_title,job_training,job_work_state,pv_unit_of_pay_9089,previous_salary,level,source_name,sector,wage_offer,wage_unit_of_pay,case_received_date_year,case_received_date_month,decision_date_month,decision_date_year,Received_Year,meansal
0,Other,PERM,A-07323-97014,NaT,Certified,J,ARMENIA,2012-02-01,NaN,Unavailable,NEW YORK,NETSOFT USA INC.,NY,NaN,NaN,NaN,NaN,NaN,NY,Year,75629.0,Level II,OES,IT,75629.00,Year,2007,NaN,2,2012,2007,75629.000
1,Other,PERM,A-07332-99439,NaT,Denied,B,POLAND,2011-12-21,NaN,Unavailable,CARLSTADT,PINNACLE ENVIRONEMNTAL CORP,NY,NaN,NaN,NaN,NaN,NaN,NY,Year,37024.0,Level I,OES,Other Economic Sector,37024.00,Year,2007,NaN,12,2011,2007,37024.000
2,Other,PERM,A-07333-99643,NaT,Certified,H-1B,INDIA,2011-12-01,NaN,Unavailable,GLEN ALLEN,"SCHNABEL ENGINEERING, INC.",VA,NaN,NaN,NaN,NaN,NaN,MD,Year,47923.0,Level I,OES,Aerospace,47923.00,Year,2007,NaN,12,2011,2007,47923.000
3,Other,PERM,A-07339-01930,NaT,Certified,B,SOUTH KOREA,2011-12-01,NaN,Unavailable,FLUSHING,EBENEZER MISSION CHURCH,NY,NaN,NaN,NaN,NaN,NaN,NY,Year,21940.0,Level II,OES,Other Economic Sector,10.97,Hour,2007,NaN,12,2011,2007,10975.485
4,Other,PERM,A-07345-03565,NaT,Certified,L,CANADA,2012-01-26,NaN,Unavailable,ALBANY,ALBANY INTERNATIONAL CORP.,NY,NaN,NaN,NaN,NaN,NaN,NY,Year,94890.0,Level IV,OES,Advanced Mfg,100000.00,Year,2007,NaN,1,2012,2007,97445.000


In [57]:
state_abb = { 'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR','California': 'CA','Colorado': 'CO',
                 'Connecticut': 'CT','Delaware': 'DE','Florida': 'FL','Georgia': 'GA','Hawaii': 'HI','Idaho': 'ID',
                  'Illinois': 'IL','Indiana': 'IN','Iowa': 'IA','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA',
                'Maine': 'ME','Maryland': 'MD', 'Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS',
                'Missouri': 'MO','Montana': 'MT','Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ',
                'New Mexico': 'NM','New York': 'NY','North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH','Oklahoma': 'OK',
                'Oregon': 'OR','Pennsylvania': 'PA','Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD',
                'Tennessee': 'TN','Texas': 'TX','Utah': 'UT','Vermont': 'VT','Virginia': 'VA','Washington': 'WA',
                'West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY','Northern Mariana Islands':'MP', 'Palau': 'PW', 
                'Puerto Rico': 'PR', 'Virgin Islands': 'VI', 'District of Columbia': 'DC','MARSHALL ISLANDS':'MH'}


us_state_abbrev = {k.upper(): v for k, v in state_abb.items()}
data['employer_state'].replace(us_state_abbrev, inplace=True)
data.employer_state = data.employer_state.astype(str)

data.employer_state.unique()

array(['NY', 'VA', 'FL', 'DE', 'MD', 'NJ', 'GA', 'IL', 'TX', 'KY', 'WY',
       'OH', 'MS', 'MA', 'NC', 'CA', 'MO', 'CT', 'AZ', 'WI', 'PA', 'WA',
       'CO', 'MN', 'NV', 'UT', 'AL', 'OK', 'LA', 'RI', 'DC', 'OR', 'MI',
       'IA', 'NH', 'IN', 'SD', 'NE', 'KS', 'TN', 'PR', 'AR', 'GU', 'WV',
       'SC', 'ID', 'VI', 'ND', 'HI', 'VT', 'ME', 'NM', 'MP', 'AK', 'MT',
       'GUAM', 'MH', 'BRITISH COLUMBIA', 'BC'], dtype=object)

### Generalizing the states

In [58]:
regions = {
    'ME':'Northeast', 
    'NY':'Northeast', 
    'NJ':'Northeast', 
    'VT':'Northeast',
    'MA':'Northeast',
    'RI':'Northeast',
    'CT':'Northeast',
    'NH':'Northeast',
    'PA':'Northeast',
     'DC':'Other',
    'IL':'Midwest',
    'IN':'Midwest',
    'IA':'Midwest', 
    'KS':'Midwest',
    'MI':'Midwest', 
    'MN':'Midwest', 
    'MO':'Midwest', 
    'NE':'Midwest',
    'ND':'Midwest', 
    'OH':'Midwest', 
    'SD':'Midwest', 
    'WI':'Midwest',
    'DE':'South',
    'FL':'South', 
    'GA':'South',
    'MD':'South',
    'NC':'South',
    'SC':'South',
    'VA':'South',
    'WV':'South',
    'AL':'South',
    'KY':'South',
    'MS':'South',
    'TN':'South',
    'AR':'South',
    'LA':'South', 
    'OK':'South',
    'TX':'South',
    'MT':'West',
    'WY':'West', 
    'CO':'West', 
    'NM':'West', 
    'ID':'West', 
    'UT':'West', 
    'AZ':'West', 
    'NV':'West',
    'WA':'West', 
    'OR':'West', 
    'CA':'West', 
    'AK':'West',
    'HI':'West', 
    'GUAM':'Island',
    'GU':'Island',
    'PR':'Island', 
    'MP':'Island', 
    'VI':'Island',
    'BC':'Northeast',
    'BRITISH COLUMBIA':'Northeast',  
    'nan':'Other',
    'FEDERATED STATES OF MICRONESIA':'Other',
'MH':'Island'}

In [59]:
def region(x):
    return regions[x]

data['employer_region'] = data['employer_state'].apply(region)

data.employer_state.replace('Other',np.nan,inplace=True)

In [60]:
data.isnull().sum()

agent_state                           0
application_type                      0
case_no                          230358
case_received_date               135271
case_status                           0
class_of_admission                    0
country_of_citizenship                0
decision_date                         0
employer_num_employees           135349
employer_yr_estab                     0
employer_city                         0
employer_name                         0
employer_state                        0
foreign_worker_info_education    135308
foreign_worker_info_major        151662
job_experience                   135281
job_title                        135313
job_training                     135284
job_work_state                        0
pv_unit_of_pay_9089                   0
previous_salary                       0
level                             27095
source_name                           0
sector                           236559
wage_offer                            0



data.agent_state.fillna('West',limit=33028,inplace=True)
data.agent_state.fillna('Northeast',limit=33026,inplace=True)
data.agent_state.fillna('South',limit=33026,inplace=True)
data.agent_state.fillna('Midwest',limit=33028,inplace=True)
data.agent_state.fillna('Island',limit=33028,inplace=True)
data.agent_state.value_counts(1)

In [61]:
data.employer_region.unique()

array(['Northeast', 'South', 'Midwest', 'West', 'Other', 'Island'],
      dtype=object)

In [62]:
data.head()

,agent_state,application_type,case_no,case_received_date,case_status,class_of_admission,country_of_citizenship,decision_date,employer_num_employees,employer_yr_estab,employer_city,employer_name,employer_state,foreign_worker_info_education,foreign_worker_info_major,job_experience,job_title,job_training,job_work_state,pv_unit_of_pay_9089,previous_salary,level,source_name,sector,wage_offer,wage_unit_of_pay,case_received_date_year,case_received_date_month,decision_date_month,decision_date_year,Received_Year,meansal,employer_region
0,Other,PERM,A-07323-97014,NaT,Certified,J,ARMENIA,2012-02-01,NaN,Unavailable,NEW YORK,NETSOFT USA INC.,NY,NaN,NaN,NaN,NaN,NaN,NY,Year,75629.0,Level II,OES,IT,75629.00,Year,2007,NaN,2,2012,2007,75629.000,Northeast
1,Other,PERM,A-07332-99439,NaT,Denied,B,POLAND,2011-12-21,NaN,Unavailable,CARLSTADT,PINNACLE ENVIRONEMNTAL CORP,NY,NaN,NaN,NaN,NaN,NaN,NY,Year,37024.0,Level I,OES,Other Economic Sector,37024.00,Year,2007,NaN,12,2011,2007,37024.000,Northeast
2,Other,PERM,A-07333-99643,NaT,Certified,H-1B,INDIA,2011-12-01,NaN,Unavailable,GLEN ALLEN,"SCHNABEL ENGINEERING, INC.",VA,NaN,NaN,NaN,NaN,NaN,MD,Year,47923.0,Level I,OES,Aerospace,47923.00,Year,2007,NaN,12,2011,2007,47923.000,South
3,Other,PERM,A-07339-01930,NaT,Certified,B,SOUTH KOREA,2011-12-01,NaN,Unavailable,FLUSHING,EBENEZER MISSION CHURCH,NY,NaN,NaN,NaN,NaN,NaN,NY,Year,21940.0,Level II,OES,Other Economic Sector,10.97,Hour,2007,NaN,12,2011,2007,10975.485,Northeast
4,Other,PERM,A-07345-03565,NaT,Certified,L,CANADA,2012-01-26,NaN,Unavailable,ALBANY,ALBANY INTERNATIONAL CORP.,NY,NaN,NaN,NaN,NaN,NaN,NY,Year,94890.0,Level IV,OES,Advanced Mfg,100000.00,Year,2007,NaN,1,2012,2007,97445.000,Northeast


In [63]:
us_state_abbrev = {k.upper(): v for k, v in state_abb.items()}
data['job_work_state'].replace(us_state_abbrev, inplace=True)
data.job_work_state = data.job_work_state.astype(str)
data.job_work_state.dtype

dtype('O')

In [64]:
def region(x):
    return regions[x]

data['job_work_state'] = data['job_work_state'].apply(region)


In [65]:
data.sector.isnull().sum()
data.employer_num_employees.isnull().sum()

135349

#### Creating a new level for missing values 

In [66]:
data.sector.fillna('unavailable',inplace=True)


In [67]:
data.sector.isnull().sum()

0

In [68]:
data.sector.value_counts(1)

unavailable              0.646995
IT                       0.143909
Advanced Mfg             0.051279
Other Economic Sector    0.043208
Finance                  0.025228
Educational Services     0.024971
Health Care              0.018667
Retail                   0.012056
Aerospace                0.011394
Hospitality              0.005268
Construction             0.003854
Automotive               0.003181
Energy                   0.002970
Transportation           0.002596
Agribusiness             0.002568
Biotechnology            0.001247
Geospatial               0.000386
Homeland Security        0.000224
Name: sector, dtype: float64

### Gernalizing Employer_Name:-

In [69]:
data.employer_name.isnull().sum() ### no null

0

In [70]:
data['employer_name'] = data['employer_name'].str.lower()
data['employer_name'].str.lower()

0                                          netsoft usa inc.
1                               pinnacle environemntal corp
2                                schnabel engineering, inc.
3                                   ebenezer mission church
4                                albany international corp.
5                               pinnacle environemntal corp
6                                       emma willard school
7                                        fds  aluminum  llc
8                       electronic data systems corporation
9                         amer brothers international, inc.
10                                              aquas, inc.
11                                nine muses and apollo inc
12                               fastpulse technology, inc.
13                                       intec billing, inc
14                                       ernest maier, inc.
15                                    niko development corp
16                           compunnel s

In [71]:
data.employer_name=data.employer_name.astype("str")   ### in str

In [72]:
data['employer_name'].nunique()  ### INC 6045

70846

### Gernalizing employer_num_employees:--

In [73]:
data.employer_num_employees.describe()

count    2.302780e+05
mean     2.326963e+04
std      5.551821e+05
min      0.000000e+00
25%      6.800000e+01
50%      1.078000e+03
75%      1.480000e+04
max      2.635506e+08
Name: employer_num_employees, dtype: float64

### Binning the number of employees to depict the size of the enterprise.

In [74]:
pd.qcut(data.employer_num_employees,q=[0.25,0.5,0.75])

0                       NaN
1                       NaN
2                       NaN
3                       NaN
4                       NaN
5                       NaN
6                       NaN
7                       NaN
8                       NaN
9                       NaN
10                      NaN
11                      NaN
12                      NaN
13                      NaN
14                      NaN
15                      NaN
16                      NaN
17                      NaN
18                      NaN
19                      NaN
20                      NaN
21                      NaN
22                      NaN
23                      NaN
24                      NaN
25                      NaN
26                      NaN
27                      NaN
28                      NaN
29                      NaN
                ...        
374332                  NaN
374333    (1078.0, 14800.0]
374334                  NaN
374335    (1078.0, 14800.0]
374336     (67.999, 

#### Creating a new level for missing values by first imputing it to 0 .

In [75]:
data.employer_num_employees.isnull().sum()
data.employer_num_employees.fillna(-1,inplace=True)

## LLC has very less employess as .A limited liability company (LLC) is a business entity that combines the limited liability protection of a business corporation with the flexible tax and organizational structure of a partnership.Because LLC have features of both corporations and partnerships, it is called a “hybrid” entity.

In [76]:
def emp(x):
    if x==-1:
        return "Unvailable"
    elif x<68 and x>=0 :
        return "Small"
    elif x<14800 and x>1078:
        return "Medium"
    else:
        return "Large"
    
data['Enterprise_Size']=data.employer_num_employees.apply(emp)


In [77]:
data['Enterprise_Size'].value_counts()

Unvailable    135349
Large         115284
Medium         57512
Small          57482
Name: Enterprise_Size, dtype: int64

In [78]:
135349/3

45116.333333333336

In [79]:
45116*3

135348

In [80]:
data.Enterprise_Size.isnull().sum()

0

In [81]:
data['Enterprise_Size'].value_counts()

Unvailable    135349
Large         115284
Medium         57512
Small          57482
Name: Enterprise_Size, dtype: int64

In [82]:
data.employer_num_employees.isnull().sum()

0

In [83]:
data.isnull().sum()[data.isnull().sum()!=0]

case_no                          230358
case_received_date               135271
foreign_worker_info_education    135308
foreign_worker_info_major        151662
job_experience                   135281
job_title                        135313
job_training                     135284
level                             27095
wage_unit_of_pay                 115689
case_received_date_month         135271
dtype: int64

In [84]:
## 135308 are null in which 7 unique categories are there so filled with prior category for increase ranging as no other info type of educaton will come rather than given so we can not do new level creation 

data.foreign_worker_info_education.fillna('unavailable',inplace=True)      ## 5 values filled with mode


In [85]:
data.foreign_worker_info_education.isnull().sum()

0

In [86]:
print(data.job_experience.value_counts(1))
data.job_experience.isnull().sum()

Y    0.558091
N    0.441909
Name: job_experience, dtype: float64


135281

In [87]:
data.job_experience.fillna('unavailable',inplace=True) #### Creating a new level for missing values 

In [88]:
data.isnull().sum()[data.isnull().sum()>0]

case_no                      230358
case_received_date           135271
foreign_worker_info_major    151662
job_title                    135313
job_training                 135284
level                         27095
wage_unit_of_pay             115689
case_received_date_month     135271
dtype: int64

In [89]:
     ##due to a lot of unique values and info already collected from sector column

In [90]:
data.job_training.fillna('unavailable',inplace=True)  #### Creating a new level for missing values 

In [91]:
   ### It is the same as pv_unit_of_pay and we already got info by mean salary

In [92]:
data.isnull().sum()[data.isnull().sum()!=0]

case_no                      230358
case_received_date           135271
foreign_worker_info_major    151662
job_title                    135313
level                         27095
wage_unit_of_pay             115689
case_received_date_month     135271
dtype: int64

In [93]:
data.level.unique()

array(['Level II', 'Level I', 'Level IV', 'Level III', nan], dtype=object)

In [94]:
6773*4

27092

In [95]:
##27095 are null value in which 4 are unique levels and no further different level will come under supervision so we decided to fill null by that prior levles

## 27095/ 4 ==6773 and 4

data.level.fillna('unavailable',inplace=True)

## filling 7% missing values

In [96]:
data.level.fillna("unavailable",inplace=True)  ### rest 3 by mode



In [97]:
data.level.isnull().sum()

0

#### The unknown level of job has the highest salary!

#### Visualizing the columns

#### We have 21k+ unique values here. They are a huge range of the employees details and has redundancy as well.

In [98]:
print(data.foreign_worker_info_major.nunique())

data.foreign_worker_info_major=data.foreign_worker_info_major.astype('str')

21851


In [99]:
strg ='ENGINEER'
for i in data.foreign_worker_info_major:
    if strg in i:
        data.foreign_worker_info_major.replace(i,'Engineer',inplace=True)


In [100]:
strng='COMPUTER'
for i in data.foreign_worker_info_major:
    if strng in i:
        data.foreign_worker_info_major.replace(i,'Computer Applications',inplace=True)
    else:
        pass

In [101]:
strng='ADMIN'
for i in data.foreign_worker_info_major:
    if strng in i:
        #print(i)
        data.foreign_worker_info_major.replace(i,'Business Administration',inplace=True)
    else:
        pass

In [102]:
str1='INFORMATION TECHNOLOGY'
for i in data.foreign_worker_info_major:
    if str1 in i:
        #print(i)
        data.foreign_worker_info_major.replace(i,'Information Technology',inplace=True)
    else:
        pass

In [103]:
str2='MEDICINE'
for i in data.foreign_worker_info_major:
    if str2 in i:
        #print(i)
        data.foreign_worker_info_major.replace(i,'Medicine',inplace=True)
    else:
        pass

In [104]:
str3='ACCOUNTING'
for i in data.foreign_worker_info_major:
    if str3 in i:
        #print(i)
        data.foreign_worker_info_major.replace(i,'Accounting',inplace=True)
    else:
        pass

In [105]:
str4='INFORMATION SYSTEM'
for i in data.foreign_worker_info_major:
    if str4 in i:
        #print(i)
        data.foreign_worker_info_major.replace(i,'Information Systems',inplace=True)
    else:
        pass

In [106]:
str5='ECONOMICS'
for i in data.foreign_worker_info_major:
    if str5 in i:
        #print(i)
        data.foreign_worker_info_major.replace(i,'Economics',inplace=True)
    else:
        pass

In [107]:
str6='MATH'
for i in data.foreign_worker_info_major:
    if str6 in i:
        #print(i)
        data.foreign_worker_info_major.replace(i,'Mathematics',inplace=True)
    else:
        pass

In [108]:
str7='PHYSICS'
for i in data.foreign_worker_info_major:
    if str7 in i:
        #print(i)
        data.foreign_worker_info_major.replace(i,'Physics',inplace=True)
    else:
        pass

In [109]:
data.foreign_worker_info_major.nunique()

9754

In [110]:
data['continent']= data.country_of_citizenship
data.continent=data.continent.replace({'ARMENIA':"ASIA",'ARMENIA':"ASIA",'INDIA':"ASIA",'SOUTH KOREA':"ASIA",
                                         'PAKISTAN':"ASIA", 'SINGAPORE':"ASIA", 'RUSSIA':"ASIA",'JAPAN':"ASIA",'CHINA':"ASIA",
                                         'PHILIPPINES':"ASIA",'TURKEY':"ASIA",'VIETNAM':"ASIA",'IRAN':"ASIA",
                                         'INDONESIA':"ASIA",'BANGLADESH':"ASIA", 'MONGOLIA':"ASIA",'ISRAEL':"ASIA",
                                         'TAIWAN':"ASIA",'NEPAL':"ASIA",'MALAYSIA':"ASIA", 'UNITED ARAB EMIRATES':"ASIA",
                                         'SRI LANKA':"ASIA",'HONG KONG':"ASIA", 'KUWAIT':"ASIA",'LEBANON':"ASIA",
                                         'JORDAN':"ASIA",'THAILAND':"ASIA",'BURKINA FASO':"ASIA",'SYRIA':"ASIA", 'LAOS':"ASIA",
                                         'CAMBODIA':"ASIA",'KYRGYZSTAN':"ASIA", 'TURKMENISTAN':"ASIA", 'BURMA (MYANMAR)':"ASIA",
                                         'AFGHANISTAN':"ASIA",'CYPRUS':"ASIA",'BHUTAN':"ASIA", 'NORTH KOREA':"ASIA",
                                         'AZERBAIJAN':"ASIA", 'OMAN':"ASIA",'BAHRAIN':"ASIA", 'KAZAKHSTAN':"ASIA",
                                         'UZBEKISTAN':"ASIA",'PALESTINE':"ASIA",'SAUDI ARABIA':"ASIA",'YEMEN':"ASIA",
                                         'IRAQ':"ASIA",'TAJIKISTAN':"ASIA",'MALDIVES':"ASIA", 'MACAU':"ASIA",
                                         'PALESTINIAN TERRITORIES':"ASIA",'QATAR':"ASIA",'BRUNEI':"ASIA"})

data.continent=data.continent.replace({'SOUTH AFRICA':"AFRICA", 'CAMEROON':"AFRICA",'MOROCCO':"AFRICA",'NIGERIA':"AFRICA",
                                         'EGYPT':"AFRICA",'KENYA':"AFRICA",'IVORY COAST':"AFRICA",'GHANA':"AFRICA",
                                         'ZAMBIA':"AFRICA",'MALI':"AFRICA",'ZIMBABWE':"AFRICA", 'MAURITIUS':"AFRICA",
                                         'SENEGAL':"AFRICA",'GUINEA':"AFRICA",'ETHIOPIA':"AFRICA",'TUNISIA':"AFRICA",
                                         'SUDAN':"AFRICA",'TANZANIA':"AFRICA",'UGANDA':"AFRICA", 'LIBYA':"AFRICA",
                                         'RWANDA':"AFRICA",'LIBERIA':"AFRICA",'SIERRA LEONE':"AFRICA",'GAMBIA':"AFRICA",
                                         'TOGO':"AFRICA",'ANGOLA':"AFRICA",'LESOTHO':"AFRICA", 'BURUNDI':"AFRICA",
                                         'GABON':"AFRICA", 'DEMOCRATIC REPUBLIC OF CONGO':"AFRICA", 
                                         'CENTRAL AFRICAN REPUBLIC':"AFRICA",'MADAGASCAR':"AFRICA",'MOZAMBIQUE':"AFRICA",
                                         'BOTSWANA':"AFRICA",'BENIN':"AFRICA",'NIGER':"AFRICA",'ALGERIA':"AFRICA",
                                         'ERITREA':"AFRICA",'MALAWI':"AFRICA", 'NAMIBIA':"AFRICA",'MAURITANIA':"AFRICA",
                                         "COTE d'IVOIRE":"AFRICA",'SAO TOME AND PRINCIPE':"AFRICA",'GUINEA-BISSAU':"AFRICA",
                                         'SEYCHELLES':"AFRICA",'SOUTH SUDAN':"AFRICA", 'EQUATORIAL GUINEA':"AFRICA",
                                         'REPUBLIC OF CONGO':"AFRICA",'SOMALIA':"AFRICA",'SWAZILAND':"AFRICA",
                                         'CAPE VERDE':"AFRICA",'CHAD':"AFRICA",'COMOROS':"AFRICA"})

data.continent = data.continent.replace({'POLAND':"EUROPE",'SWEDEN':"EUROPE", 'PORTUGAL':"EUROPE",'SPAIN':"EUROPE",
                                           'AUSTRIA':"EUROPE",'ITALY':"EUROPE", 'IRELAND':"EUROPE", 'BULGARIA':"EUROPE",
                                           'UNITED KINGDOM':"EUROPE",'SLOVAKIA':"EUROPE",'ROMANIA':"EUROPE", 'UKRAINE':"EUROPE",
                                           'HUNGARY':"EUROPE", 'MACEDONIA':"EUROPE",'GERMANY':"EUROPE",'MOLDOVA':"EUROPE",
                                           'NETHERLANDS':"EUROPE",'GREECE':"EUROPE",'SERBIA AND MONTENEGRO':"EUROPE",
                                           'SWITZERLAND':"EUROPE",'SLOVENIA':"EUROPE",'BELARUS':"EUROPE",
                                           'CZECH REPUBLIC':"EUROPE",'BELGIUM':"EUROPE",'ALBANIA':"EUROPE",'ESTONIA':"EUROPE",
                                           'DENMARK':"EUROPE",'NORWAY':"EUROPE",'LITHUANIA':"EUROPE", 'FINLAND':"EUROPE",
                                           'LATVIA':"EUROPE",'BOSNIA AND HERZEGOVINA':"EUROPE",'YUGOSLAVIA':"EUROPE",
                                           'CROATIA':"EUROPE",'MALTA':"EUROPE",'ICELAND':"EUROPE", 'KOSOVO':"EUROPE",
                                           'LUXEMBOURG':"EUROPE",'SERBIA':"EUROPE",'MONTENEGRO':"EUROPE",'MONACO':"EUROPE",
                                           'LIECHTENSTEIN':"EUROPE",'ANDORRA':"EUROPE",'SOVIET UNION':"EUROPE"})


data.continent = data.continent.replace({'CANADA':"North America",'MEXICO':"North America",'VENEZUELA':"North America",
                                           'EL SALVADOR':"North America", 'JAMAICA':"North America", 
                                           'TRINIDAD AND TOBAGO':"North America",'GUATEMALA':"North America",
                                           'UNITED STATES OF AMERICA':"North America",'BOLIVIA':"North America",
                                           'DOMINICAN REPUBLIC':"North America", 'FRANCE':"North America",
                                           'PANAMA':"North America",'COSTA RICA':"North America",'BARBADOS':"North America", 
                                           'GRENADA':"North America", 'HONDURAS':"North America",'BELIZE':"North America",
                                           'DOMINICA':"North America",'ST VINCENT':"North America", 'NICARAGUA':"North America",
                                           'ST LUCIA':"North America",'URUGUAY':"North America",'BAHAMAS':"North America",
                                           'ST KITTS AND NEVIS':"North America", 'HAITI':"North America",
                                           'CAYMAN ISLANDS':"North America", 'BERMUDA':"North America",'ARUBA':"North America",
                                           'TURKS AND CAICOS ISLANDS':"North America",'ANTIGUA AND BARBUDA':"North America",
                                           'SAINT VINCENT AND THE GRENADINES':"North America",
                                           'BRITISH VIRGIN ISLANDS':"North America",'CUBA':"North America",'SINT MAARTEN':"North America"})

data.continent = data.continent.replace({'ECUADOR':"South America",'COLOMBIA':"South America",'BRAZIL':"South America",
                                           'ARGENTINA':"South America", 'PERU':"South America",'PARAGUAY':"South America",
                                           'CHILE':"South America",'GEORGIA':"South America",'GUYANA':"South America",
                                           'SURINAME':"South America", 'NETHERLANDS ANTILLES':"South America"})

data.continent = data.continent.replace({'AUSTRALIA':"Oceania",'NEW ZEALAND':"Oceania",'FIJI':"Oceania",
                                           'PAPUA NEW GUINEA':"Oceania",'VANUATU':"Oceania",'MARSHALL ISLANDS':"Oceania",
                                           'KIRIBATI':"Oceania",'SAMOA':"Oceania"})


In [111]:
data.continent.value_counts()

ASIA             294092
North America     34066
EUROPE            21811
South America      8239
AFRICA             5305
Oceania            2114
Name: continent, dtype: int64

In [112]:
pd.crosstab(data.case_status,data.source_name)

source_name,CBA,DBA,Employer Conducted,OES,Other,SCA
case_status,,,,,,
Certified,1472,0,117,163531,12866,14
Certified-Expired,1285,0,94,133152,10327,8
Denied,629,24,296,22846,1229,45
Withdrawn,198,0,26,16403,1057,8


In [113]:
data.drop('employer_city',axis=1,inplace=True)    # 8.3k unique vales

def draw_histograms(dataframe, features, rows, cols):
    fig=plt.figure(figsize=(20,20))
    for i, feature in enumerate(features):
        ax=fig.add_subplot(rows,cols,i+1)
        dataframe[feature].hist(bins=20,ax=ax,facecolor='midnightblue')
        ax.set_title(feature+" Distribution",color='DarkRed')
        
    fig.tight_layout()  
    plt.show()
    
    
draw_histograms(data,data.columns,10,3)

### Exporting Clean Data

In [114]:
data.employer_name.nunique()

70846

In [115]:
data.drop(['country_of_citizenship'],axis=1,inplace=True)

In [116]:
data.head()

,agent_state,application_type,case_no,case_received_date,case_status,class_of_admission,decision_date,employer_num_employees,employer_yr_estab,employer_name,employer_state,foreign_worker_info_education,foreign_worker_info_major,job_experience,job_title,job_training,job_work_state,pv_unit_of_pay_9089,previous_salary,level,source_name,sector,wage_offer,wage_unit_of_pay,case_received_date_year,case_received_date_month,decision_date_month,decision_date_year,Received_Year,meansal,employer_region,Enterprise_Size,continent
0,Other,PERM,A-07323-97014,NaT,Certified,J,2012-02-01,-1.0,Unavailable,netsoft usa inc.,NY,unavailable,nan,unavailable,NaN,unavailable,Northeast,Year,75629.0,Level II,OES,IT,75629.00,Year,2007,NaN,2,2012,2007,75629.000,Northeast,Unvailable,ASIA
1,Other,PERM,A-07332-99439,NaT,Denied,B,2011-12-21,-1.0,Unavailable,pinnacle environemntal corp,NY,unavailable,nan,unavailable,NaN,unavailable,Northeast,Year,37024.0,Level I,OES,Other Economic Sector,37024.00,Year,2007,NaN,12,2011,2007,37024.000,Northeast,Unvailable,EUROPE
2,Other,PERM,A-07333-99643,NaT,Certified,H-1B,2011-12-01,-1.0,Unavailable,"schnabel engineering, inc.",VA,unavailable,nan,unavailable,NaN,unavailable,South,Year,47923.0,Level I,OES,Aerospace,47923.00,Year,2007,NaN,12,2011,2007,47923.000,South,Unvailable,ASIA
3,Other,PERM,A-07339-01930,NaT,Certified,B,2011-12-01,-1.0,Unavailable,ebenezer mission church,NY,unavailable,nan,unavailable,NaN,unavailable,Northeast,Year,21940.0,Level II,OES,Other Economic Sector,10.97,Hour,2007,NaN,12,2011,2007,10975.485,Northeast,Unvailable,ASIA
4,Other,PERM,A-07345-03565,NaT,Certified,L,2012-01-26,-1.0,Unavailable,albany international corp.,NY,unavailable,nan,unavailable,NaN,unavailable,Northeast,Year,94890.0,Level IV,OES,Advanced Mfg,100000.00,Year,2007,NaN,1,2012,2007,97445.000,Northeast,Unvailable,North America


In [117]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365627 entries, 0 to 374361
Data columns (total 33 columns):
agent_state                      365627 non-null object
application_type                 365627 non-null object
case_no                          135269 non-null object
case_received_date               230356 non-null datetime64[ns]
case_status                      365627 non-null object
class_of_admission               365627 non-null object
decision_date                    365627 non-null datetime64[ns]
employer_num_employees           365627 non-null float32
employer_yr_estab                365627 non-null object
employer_name                    365627 non-null object
employer_state                   365627 non-null object
foreign_worker_info_education    365627 non-null object
foreign_worker_info_major        365627 non-null object
job_experience                   365627 non-null object
job_title                        230314 non-null object
job_training                    

### Data Preprocessing:-

In [118]:
data.isnull().sum()[data.isnull().sum()!=0]

case_no                     230358
case_received_date          135271
job_title                   135313
wage_unit_of_pay            115689
case_received_date_month    135271
dtype: int64

In [119]:
data.columns

Index(['agent_state', 'application_type', 'case_no', 'case_received_date',
       'case_status', 'class_of_admission', 'decision_date',
       'employer_num_employees', 'employer_yr_estab', 'employer_name',
       'employer_state', 'foreign_worker_info_education',
       'foreign_worker_info_major', 'job_experience', 'job_title',
       'job_training', 'job_work_state', 'pv_unit_of_pay_9089',
       'previous_salary', 'level', 'source_name', 'sector', 'wage_offer',
       'wage_unit_of_pay', 'case_received_date_year',
       'case_received_date_month', 'decision_date_month', 'decision_date_year',
       'Received_Year', 'meansal', 'employer_region', 'Enterprise_Size',
       'continent'],
      dtype='object')

In [120]:
data.drop(columns=['case_no','job_title','wage_unit_of_pay'],inplace=True)

In [121]:
data.shape

(365627, 30)

In [122]:
data.isnull().sum()[data.isnull().sum()!=0]

case_received_date          135271
case_received_date_month    135271
dtype: int64

In [123]:
data['duration_year']=data.decision_date_year-data.case_received_date_year

In [124]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 365627 entries, 0 to 374361
Data columns (total 31 columns):
agent_state                      365627 non-null object
application_type                 365627 non-null object
case_received_date               230356 non-null datetime64[ns]
case_status                      365627 non-null object
class_of_admission               365627 non-null object
decision_date                    365627 non-null datetime64[ns]
employer_num_employees           365627 non-null float32
employer_yr_estab                365627 non-null object
employer_name                    365627 non-null object
employer_state                   365627 non-null object
foreign_worker_info_education    365627 non-null object
foreign_worker_info_major        365627 non-null object
job_experience                   365627 non-null object
job_training                     365627 non-null object
job_work_state                   365627 non-null object
pv_unit_of_pay_9089             

In [125]:


#data.drop('foreign_worker_info_major',axis=1,inplace=True)
#data.drop('employer_state',axis=1,inplace=True)

data.drop(['decision_date'],axis=1,inplace=True)
#data.drop('agent_state',axis=1,inplace=True)
data.drop(['previous_salary','wage_offer'],axis=1,inplace=True)
#data.drop('job_title',axis=1,inplace=True)
#data.drop('wage_unit_of_pay',axis=1,inplace=True)
data.drop('case_received_date',axis=1,inplace=True)

In [126]:
data.drop(['case_received_date_year','decision_date_year'],axis=1,inplace=True)

In [127]:
data.drop('Received_Year',axis=1,inplace=True)

In [128]:
data.drop('employer_region',axis=1,inplace=True)

In [129]:
data.columns

Index(['agent_state', 'application_type', 'case_status', 'class_of_admission',
       'employer_num_employees', 'employer_yr_estab', 'employer_name',
       'employer_state', 'foreign_worker_info_education',
       'foreign_worker_info_major', 'job_experience', 'job_training',
       'job_work_state', 'pv_unit_of_pay_9089', 'level', 'source_name',
       'sector', 'case_received_date_month', 'decision_date_month', 'meansal',
       'Enterprise_Size', 'continent', 'duration_year'],
      dtype='object')

In [130]:
data.isnull().sum()[data.isnull().sum()!=0]

case_received_date_month    135271
dtype: int64

In [131]:
data.head()

,agent_state,application_type,case_status,class_of_admission,employer_num_employees,employer_yr_estab,employer_name,employer_state,foreign_worker_info_education,foreign_worker_info_major,job_experience,job_training,job_work_state,pv_unit_of_pay_9089,level,source_name,sector,case_received_date_month,decision_date_month,meansal,Enterprise_Size,continent,duration_year
0,Other,PERM,Certified,J,-1.0,Unavailable,netsoft usa inc.,NY,unavailable,nan,unavailable,unavailable,Northeast,Year,Level II,OES,IT,NaN,2,75629.000,Unvailable,ASIA,5
1,Other,PERM,Denied,B,-1.0,Unavailable,pinnacle environemntal corp,NY,unavailable,nan,unavailable,unavailable,Northeast,Year,Level I,OES,Other Economic Sector,NaN,12,37024.000,Unvailable,EUROPE,4
2,Other,PERM,Certified,H-1B,-1.0,Unavailable,"schnabel engineering, inc.",VA,unavailable,nan,unavailable,unavailable,South,Year,Level I,OES,Aerospace,NaN,12,47923.000,Unvailable,ASIA,4
3,Other,PERM,Certified,B,-1.0,Unavailable,ebenezer mission church,NY,unavailable,nan,unavailable,unavailable,Northeast,Year,Level II,OES,Other Economic Sector,NaN,12,10975.485,Unvailable,ASIA,4
4,Other,PERM,Certified,L,-1.0,Unavailable,albany international corp.,NY,unavailable,nan,unavailable,unavailable,Northeast,Year,Level IV,OES,Advanced Mfg,NaN,1,97445.000,Unvailable,North America,5


In [132]:
data.shape

(365627, 23)

In [133]:
data.columns

Index(['agent_state', 'application_type', 'case_status', 'class_of_admission',
       'employer_num_employees', 'employer_yr_estab', 'employer_name',
       'employer_state', 'foreign_worker_info_education',
       'foreign_worker_info_major', 'job_experience', 'job_training',
       'job_work_state', 'pv_unit_of_pay_9089', 'level', 'source_name',
       'sector', 'case_received_date_month', 'decision_date_month', 'meansal',
       'Enterprise_Size', 'continent', 'duration_year'],
      dtype='object')

In [134]:
data.isnull().sum()

agent_state                           0
application_type                      0
case_status                           0
class_of_admission                    0
employer_num_employees                0
employer_yr_estab                     0
employer_name                         0
employer_state                        0
foreign_worker_info_education         0
foreign_worker_info_major             0
job_experience                        0
job_training                          0
job_work_state                        0
pv_unit_of_pay_9089                   0
level                                 0
source_name                           0
sector                                0
case_received_date_month         135271
decision_date_month                   0
meansal                               0
Enterprise_Size                       0
continent                             0
duration_year                         0
dtype: int64

In [135]:
data.drop(['case_received_date_month','decision_date_month'],axis=1,inplace=True)

In [136]:
#sns.pairplot(data,hue='case_status')

## Statistic

In [137]:
### statistical significant with Target Variable.

## numerical columns:- meansal & Case status
from scipy.stats import shapiro
import scipy.stats as st

print('ttestvale',st.ttest_ind(data.meansal,data.case_status))

print('shapiro value of meansal=',shapiro(data.meansal),'kurtosis=',data.meansal.kurt(),'skew=',data.meansal.skew())
sns.distplot(meansal)
plt.show()

print('ttestvale',st.ttest_ind(data.decision_date_year,data.case_status))
print('shapiro value of decision_date_year=',shapiro(data.decision_date_year),'kurtosis=',data.decision_date_year.kurt(),'skew=',data.decision_date_year.skew())
sns.distplot(decision_date_year)
plt.show()

print('ttestvale',st.ttest_ind(data.case_received_date_year,data.case_status))

print('shapiro value of case_received_date_year=',shapiro(data.case_received_date_year),'kurtosis=',data.case_received_date_year.kurt(),'skew=',data.case_received_date_year.skew())
sns.distplot(case_received_date_year)
plt.show()

In [138]:
data.columns


Index(['agent_state', 'application_type', 'case_status', 'class_of_admission',
       'employer_num_employees', 'employer_yr_estab', 'employer_name',
       'employer_state', 'foreign_worker_info_education',
       'foreign_worker_info_major', 'job_experience', 'job_training',
       'job_work_state', 'pv_unit_of_pay_9089', 'level', 'source_name',
       'sector', 'meansal', 'Enterprise_Size', 'continent', 'duration_year'],
      dtype='object')

In [139]:
data.shape

(365627, 21)

In [140]:
data.drop('pv_unit_of_pay_9089',axis=1,inplace=True)

In [141]:
data.shape

(365627, 20)

In [142]:
data.head()

,agent_state,application_type,case_status,class_of_admission,employer_num_employees,employer_yr_estab,employer_name,employer_state,foreign_worker_info_education,foreign_worker_info_major,job_experience,job_training,job_work_state,level,source_name,sector,meansal,Enterprise_Size,continent,duration_year
0,Other,PERM,Certified,J,-1.0,Unavailable,netsoft usa inc.,NY,unavailable,nan,unavailable,unavailable,Northeast,Level II,OES,IT,75629.000,Unvailable,ASIA,5
1,Other,PERM,Denied,B,-1.0,Unavailable,pinnacle environemntal corp,NY,unavailable,nan,unavailable,unavailable,Northeast,Level I,OES,Other Economic Sector,37024.000,Unvailable,EUROPE,4
2,Other,PERM,Certified,H-1B,-1.0,Unavailable,"schnabel engineering, inc.",VA,unavailable,nan,unavailable,unavailable,South,Level I,OES,Aerospace,47923.000,Unvailable,ASIA,4
3,Other,PERM,Certified,B,-1.0,Unavailable,ebenezer mission church,NY,unavailable,nan,unavailable,unavailable,Northeast,Level II,OES,Other Economic Sector,10975.485,Unvailable,ASIA,4
4,Other,PERM,Certified,L,-1.0,Unavailable,albany international corp.,NY,unavailable,nan,unavailable,unavailable,Northeast,Level IV,OES,Advanced Mfg,97445.000,Unvailable,North America,5


In [143]:
data.drop('employer_name',axis=1,inplace=True)

In [144]:
data.drop('foreign_worker_info_major',axis=1,inplace=True)

In [145]:
data.class_of_admission.nunique()

30

In [146]:
### exporting cleaan data

#data.to_csv(r'C:\Users\hp\Desktop\Capstone Project\visa process\UsVisaDATASET.csv')

### Base Model

In [147]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
sc=StandardScaler()

In [148]:
def target(x):
    if x=="Certified":
        return 0
    
    elif x=="Certified-Expired":
        return 1
    elif x=='Denied':
        return 2
    else:
        return 3
data.case_status=data.case_status.apply(target)

In [165]:
data.head()

,agent_state,application_type,case_status,class_of_admission,employer_num_employees,employer_yr_estab,employer_state,foreign_worker_info_education,job_experience,job_training,job_work_state,level,source_name,sector,meansal,Enterprise_Size,continent,duration_year
0,Other,PERM,0,J,-1.0,Unavailable,NY,unavailable,unavailable,unavailable,Northeast,Level II,OES,IT,75629.000,Unvailable,ASIA,5
1,Other,PERM,2,B,-1.0,Unavailable,NY,unavailable,unavailable,unavailable,Northeast,Level I,OES,Other Economic Sector,37024.000,Unvailable,EUROPE,4
2,Other,PERM,0,H-1B,-1.0,Unavailable,VA,unavailable,unavailable,unavailable,South,Level I,OES,Aerospace,47923.000,Unvailable,ASIA,4
3,Other,PERM,0,B,-1.0,Unavailable,NY,unavailable,unavailable,unavailable,Northeast,Level II,OES,Other Economic Sector,10975.485,Unvailable,ASIA,4
4,Other,PERM,0,L,-1.0,Unavailable,NY,unavailable,unavailable,unavailable,Northeast,Level IV,OES,Advanced Mfg,97445.000,Unvailable,North America,5


In [149]:
dummy=pd.get_dummies(data,drop_first=True)

In [150]:
dummy.head()

,case_status,employer_num_employees,meansal,duration_year,agent_state_Midwest,agent_state_Northeast,agent_state_Other,agent_state_South,agent_state_West,application_type_ONLINE,application_type_PERM,application_type_unavailable,class_of_admission_AOS,class_of_admission_B,class_of_admission_C,class_of_admission_D-1,class_of_admission_E,class_of_admission_EWI,class_of_admission_F,class_of_admission_G,class_of_admission_H-1B,class_of_admission_I,class_of_admission_J,class_of_admission_K-1,class_of_admission_L,...,sector_Aerospace,sector_Agribusiness,sector_Automotive,sector_Biotechnology,sector_Construction,sector_Educational Services,sector_Energy,sector_Finance,sector_Geospatial,sector_Health Care,sector_Homeland Security,sector_Hospitality,sector_IT,sector_Other Economic Sector,sector_Retail,sector_Transportation,sector_unavailable,Enterprise_Size_Medium,Enterprise_Size_Small,Enterprise_Size_Unvailable,continent_ASIA,continent_EUROPE,continent_North America,continent_Oceania,continent_South America
0,0,-1.0,75629.000,5,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0
1,2,-1.0,37024.000,4,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0
2,0,-1.0,47923.000,4,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
3,0,-1.0,10975.485,4,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0
4,0,-1.0,97445.000,5,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0


In [151]:
x=dummy.drop('case_status',axis=1)
y=dummy['case_status']

In [152]:
x.head()

,employer_num_employees,meansal,duration_year,agent_state_Midwest,agent_state_Northeast,agent_state_Other,agent_state_South,agent_state_West,application_type_ONLINE,application_type_PERM,application_type_unavailable,class_of_admission_AOS,class_of_admission_B,class_of_admission_C,class_of_admission_D-1,class_of_admission_E,class_of_admission_EWI,class_of_admission_F,class_of_admission_G,class_of_admission_H-1B,class_of_admission_I,class_of_admission_J,class_of_admission_K-1,class_of_admission_L,class_of_admission_M,...,sector_Aerospace,sector_Agribusiness,sector_Automotive,sector_Biotechnology,sector_Construction,sector_Educational Services,sector_Energy,sector_Finance,sector_Geospatial,sector_Health Care,sector_Homeland Security,sector_Hospitality,sector_IT,sector_Other Economic Sector,sector_Retail,sector_Transportation,sector_unavailable,Enterprise_Size_Medium,Enterprise_Size_Small,Enterprise_Size_Unvailable,continent_ASIA,continent_EUROPE,continent_North America,continent_Oceania,continent_South America
0,-1.0,75629.000,5,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0
1,-1.0,37024.000,4,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0
2,-1.0,47923.000,4,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
3,-1.0,10975.485,4,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0
4,-1.0,97445.000,5,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0


In [153]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=0)

In [154]:
scaled_train=pd.DataFrame(sc.fit_transform(xtrain),columns=xtrain.columns)
scaled_test=pd.DataFrame(sc.transform(xtest),columns=xtest.columns)
scaled_train.head()

,employer_num_employees,meansal,duration_year,agent_state_Midwest,agent_state_Northeast,agent_state_Other,agent_state_South,agent_state_West,application_type_ONLINE,application_type_PERM,application_type_unavailable,class_of_admission_AOS,class_of_admission_B,class_of_admission_C,class_of_admission_D-1,class_of_admission_E,class_of_admission_EWI,class_of_admission_F,class_of_admission_G,class_of_admission_H-1B,class_of_admission_I,class_of_admission_J,class_of_admission_K-1,class_of_admission_L,class_of_admission_M,...,sector_Aerospace,sector_Agribusiness,sector_Automotive,sector_Biotechnology,sector_Construction,sector_Educational Services,sector_Energy,sector_Finance,sector_Geospatial,sector_Health Care,sector_Homeland Security,sector_Hospitality,sector_IT,sector_Other Economic Sector,sector_Retail,sector_Transportation,sector_unavailable,Enterprise_Size_Medium,Enterprise_Size_Small,Enterprise_Size_Unvailable,continent_ASIA,continent_EUROPE,continent_North America,continent_Oceania,continent_South America
0,-0.028384,-0.060954,-0.848515,-0.290446,-0.411846,1.103059,-0.413111,-0.467741,1.500621,-0.24343,-1.307056,0.0,-0.10395,-0.011695,-0.00442,-0.125603,-0.072901,-0.213203,-0.022369,0.552883,-0.015313,-0.053885,-0.002795,-0.240626,-0.008616,...,-0.107036,-0.05169,-0.056451,-0.03488,-0.061328,-0.159698,-0.053262,-0.162088,-0.019869,-0.137632,-0.014255,-0.072847,2.435694,-0.210475,-0.110439,-0.051117,-1.355839,-0.432095,-0.432661,1.306333,0.493467,-0.251091,-0.321107,-0.076781,-0.151762
1,-0.028384,-0.059375,-0.848515,-0.290446,-0.411846,1.103059,-0.413111,-0.467741,1.500621,-0.24343,-1.307056,0.0,-0.10395,-0.011695,-0.00442,-0.125603,-0.072901,-0.213203,-0.022369,0.552883,-0.015313,-0.053885,-0.002795,-0.240626,-0.008616,...,-0.107036,-0.05169,-0.056451,-0.03488,-0.061328,-0.159698,-0.053262,6.169488,-0.019869,-0.137632,-0.014255,-0.072847,-0.410561,-0.210475,-0.110439,-0.051117,-1.355839,-0.432095,-0.432661,1.306333,-2.026476,-0.251091,-0.321107,-0.076781,6.589253
2,-0.016248,-0.055978,0.536648,-0.290446,-0.411846,1.103059,-0.413111,-0.467741,-0.666391,-0.24343,0.765078,0.0,-0.10395,-0.011695,-0.00442,-0.125603,-0.072901,-0.213203,-0.022369,0.552883,-0.015313,-0.053885,-0.002795,-0.240626,-0.008616,...,-0.107036,-0.05169,-0.056451,-0.03488,-0.061328,-0.159698,-0.053262,-0.162088,-0.019869,-0.137632,-0.014255,-0.072847,-0.410561,-0.210475,-0.110439,-0.051117,0.737551,2.314308,-0.432661,-0.765502,0.493467,-0.251091,-0.321107,-0.076781,-0.151762
3,-0.028384,-0.058726,-0.848515,-0.290446,-0.411846,1.103059,-0.413111,-0.467741,1.500621,-0.24343,-1.307056,0.0,-0.10395,-0.011695,-0.00442,-0.125603,-0.072901,-0.213203,-0.022369,0.552883,-0.015313,-0.053885,-0.002795,-0.240626,-0.008616,...,-0.107036,-0.05169,-0.056451,-0.03488,-0.061328,-0.159698,-0.053262,-0.162088,-0.019869,-0.137632,-0.014255,-0.072847,-0.410561,4.751163,-0.110439,-0.051117,-1.355839,-0.432095,-0.432661,1.306333,0.493467,-0.251091,-0.321107,-0.076781,-0.151762
4,-0.028384,-0.059693,-0.848515,-0.290446,-0.411846,1.103059,-0.413111,-0.467741,1.500621,-0.24343,-1.307056,0.0,-0.10395,-0.011695,-0.00442,-0.125603,-0.072901,-0.213203,-0.022369,0.552883,-0.015313,-0.053885,-0.002795,-0.240626,-0.008616,...,-0.107036,-0.05169,-0.056451,-0.03488,-0.061328,-0.159698,-0.053262,-0.162088,-0.019869,-0.137632,-0.014255,-0.072847,-0.410561,-0.210475,-0.110439,-0.051117,-1.355839,-0.432095,-0.432661,1.306333,0.493467,-0.251091,-0.321107,-0.076781,-0.151762


In [155]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()

In [156]:
model=rf.fit(scaled_train,ytrain)
ypred=model.predict(scaled_test)

In [157]:
from sklearn.metrics import classification_report,f1_score,confusion_matrix,cohen_kappa_score,matthews_corrcoef,accuracy_score,roc_auc_score

In [158]:
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.65      0.70      0.67     53111
           1       0.61      0.62      0.61     43642
           2       0.58      0.40      0.47      7584
           3       0.24      0.12      0.16      5352

    accuracy                           0.62    109689
   macro avg       0.52      0.46      0.48    109689
weighted avg       0.61      0.62      0.61    109689



In [159]:
rf.score(xtrain,ytrain)

0.4357578788612867

In [160]:
accuracy_score(ytest,ypred)

0.6183847058501764

In [161]:
cohen_kappa_score(ytest,ypred)

0.3429946576728651

In [162]:
matthews_corrcoef(ytest,ypred)

0.34405039542954385

In [163]:
''

''